In [55]:
import os
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.schema import HumanMessage, AIMessage, BaseChatMessageHistory
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory


In [56]:
def run_chatbot():
    key = os.getenv("GROQ_API_KEY")
    llm = ChatGroq(
        groq_api_key = key,
        model="mixtral-8x7b-32768",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
    system_message = SystemMessagePromptTemplate.from_template(
        "You are a helpful assistant. Analyze the following query and provide a response with context. Do not be too verbose. Give a short and brief answer. Store memory of previous question."
    )
    human_message = HumanMessagePromptTemplate.from_template("{query}")
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    llm_chain = LLMChain(
        prompt=chat_prompt,
        llm=llm,
    )
    def chatbot(query):
        response = llm_chain.run({"query": query})
        return response
    store = {}
    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]
    chain = llm_chain  
    
    with_message_history = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="query",
    )
    session_id = "abc20" 
    config = {"configurable": {"session_id": session_id}}
    while True:
        user_query = input("You: ")
        if user_query.strip().lower() == "bye":
            print("See you next time!")
            break
        history = get_session_history(session_id)
        response = with_message_history.invoke(
            {
                "query": [HumanMessage(content=user_query)],
            },
            config=config,
        )
        if 'text' in response:
            history.add_message(AIMessage(content=response['text']))
            print("Assistant:", response['text'])
        else:
            print("Assistant: Sorry, I don't understand.")
        
       
if __name__ == "__main__":
    run_chatbot()

See you next time!
